In [1]:
import glob
import os
import librosa
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew


featureVectorLength = 140  # n_mfcc * number_of_summary_statistics

def extractFeaturesFromWaveform(file_name):
    rawData, sampleRate = librosa.load(file_name)

    # one row per extracted coefficient, one column per frame
    mfccs = librosa.feature.mfcc(y=rawData, sr=sampleRate, n_mfcc=20)

    mfccs_min = np.min(mfccs, axis=1)  # row-wise summaries
    mfccs_max = np.max(mfccs, axis=1)
    mfccs_median = np.median(mfccs, axis=1)
    mfccs_mean = np.mean(mfccs, axis=1)
    mfccs_variance = np.var(mfccs, axis=1)
    mfccs_skeweness = skew(mfccs, axis=1)
    mfccs_kurtosis = kurtosis(mfccs, axis=1)

    return mfccs_min, mfccs_max, mfccs_median, mfccs_mean, mfccs_variance, mfccs_skeweness, mfccs_kurtosis

def extract_features_from_directories(parentDirectory, sub_dirs, file_ext="*.wav"):
    feature_matrix, labels = np.empty((0, featureVectorLength)), np.empty(0)

    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parentDirectory, sub_dir, file_ext)):
            try:
                mfccs_min, mfccs_max, mfccs_median, mfccs_mean, mfccs_variance, mfccs_skeweness, mfccs_kurtosis = extractFeaturesFromWaveform(fn)
                print("Finished processing file: ", fn)
            except Exception as e:
                print("Error while processing file: ", fn)
                continue

            # concatenate extracted features
            new_feature_vector = np.hstack([mfccs_min, mfccs_max, mfccs_median, mfccs_mean, mfccs_variance, mfccs_skeweness, mfccs_kurtosis])

            # add current feature vector as last row in feature matrix
            feature_matrix = np.vstack([feature_matrix, new_feature_vector])

            # extracts label from the file name. Change '\\' to  '/' on Unix systems
            labels = np.append(labels, fn.split('/')[2].split('-')[1])

    return np.array(feature_matrix), np.array(labels, dtype=np.int)

def prepareFeatures(trainingDirectory, validationDirectory, trainingName, validationName):

    parentDirectory = 'Sound'  # name of the directory which contains the recordings
    trainingSubDirectories = trainingDirectory
    validationSubDirectories = validationDirectory

    # ndarrays
    training_features, training_labels = extract_features_from_directories(parentDirectory, trainingSubDirectories)
    test_features, test_labels = extract_features_from_directories(parentDirectory, validationSubDirectories)

    # convert ndarray to pandas dataframe
    training_examples = pd.DataFrame(training_features, columns=list(range(1, featureVectorLength+1)))
    # convert ndarray to pandas series
    training_labels = pd.Series(training_labels.tolist())

    # convert ndarray to pandas dataframe
    validation_examples = pd.DataFrame(test_features, columns=list(range(1, featureVectorLength+1)))
    # convert ndarray to pandas series
    validation_labels = pd.Series(test_labels.tolist())

    # store extracted training data
    training_examples.to_pickle('Extracted_Features-' + trainingName + '_features.pkl')
    training_labels.to_pickle('Extracted_Features-' + trainingName + '_labels.pkl')

    # store extracted validation data
    validation_examples.to_pickle('Extracted_Features-' + validationName + '_features.pkl')
    validation_labels.to_pickle('Extracted_Features-' + validationName + '_labels.pkl')


# First 9 folds will be used for training, the tenth for validation.
trainingDirectory = ["fold1", "fold2", "fold3", "fold4", "fold5", "fold6", "fold7", "fold8", "fold9"]
validationDirectory = ["fold10"]
prepareFeatures(trainingDirectory, validationDirectory, 'notFold10', 'fold10')

# Read the stored features and labels:
print(pd.read_pickle('Extracted_Features-fold10_features.pkl')) 
print(pd.read_pickle('Extracted_Features-fold10_labels.pkl'))

Finished processing file:  Sound/fold1/135776-2-0-49.wav
Finished processing file:  Sound/fold1/46654-6-0-0.wav
Finished processing file:  Sound/fold1/57320-0-0-24.wav
Finished processing file:  Sound/fold1/134717-0-0-26.wav
Finished processing file:  Sound/fold1/174276-7-5-0.wav
Finished processing file:  Sound/fold1/180937-7-2-6.wav
Finished processing file:  Sound/fold1/17913-4-0-1.wav
Finished processing file:  Sound/fold1/103074-7-4-6.wav
Finished processing file:  Sound/fold1/176787-5-0-9.wav
Finished processing file:  Sound/fold1/55020-4-0-11.wav
Finished processing file:  Sound/fold1/138015-3-0-7.wav
Finished processing file:  Sound/fold1/180937-7-0-4.wav
Finished processing file:  Sound/fold1/180937-7-1-13.wav
Finished processing file:  Sound/fold1/157867-8-0-10.wav
Finished processing file:  Sound/fold1/103074-7-2-0.wav
Finished processing file:  Sound/fold1/113205-5-1-4.wav
Finished processing file:  Sound/fold1/57584-4-0-8.wav
Finished processing file:  Sound/fold1/118279-8

Finished processing file:  Sound/fold1/99180-9-0-36.wav
Finished processing file:  Sound/fold1/134717-0-0-21.wav
Finished processing file:  Sound/fold1/57320-0-0-37.wav
Finished processing file:  Sound/fold1/180937-7-2-1.wav
Finished processing file:  Sound/fold1/162540-1-2-0.wav
Finished processing file:  Sound/fold1/57553-3-0-0.wav
Finished processing file:  Sound/fold1/138031-2-0-23.wav
Finished processing file:  Sound/fold1/174294-6-2-0.wav
Finished processing file:  Sound/fold1/72579-3-0-3.wav
Finished processing file:  Sound/fold1/57320-0-0-21.wav
Finished processing file:  Sound/fold1/180937-7-2-3.wav
Finished processing file:  Sound/fold1/72261-3-0-27.wav
Finished processing file:  Sound/fold1/134717-0-0-23.wav
Finished processing file:  Sound/fold1/197318-6-5-0.wav
Finished processing file:  Sound/fold1/146186-5-0-8.wav
Finished processing file:  Sound/fold1/162540-1-0-0.wav
Finished processing file:  Sound/fold1/160094-3-0-0.wav
Finished processing file:  Sound/fold1/57553-3-

Finished processing file:  Sound/fold1/176787-5-0-25.wav
Finished processing file:  Sound/fold1/77766-9-0-7.wav
Finished processing file:  Sound/fold1/194732-9-0-41.wav
Finished processing file:  Sound/fold1/176787-5-0-19.wav
Finished processing file:  Sound/fold1/165645-4-0-0.wav
Finished processing file:  Sound/fold1/164053-8-2-2.wav
Finished processing file:  Sound/fold1/78360-4-0-8.wav
Finished processing file:  Sound/fold1/44736-5-0-1.wav
Finished processing file:  Sound/fold1/73277-9-0-16.wav
Finished processing file:  Sound/fold1/193698-2-0-113.wav
Finished processing file:  Sound/fold1/180937-7-1-4.wav
Finished processing file:  Sound/fold1/124489-9-0-14.wav
Finished processing file:  Sound/fold1/165067-2-0-72.wav
Finished processing file:  Sound/fold1/7061-6-0-0.wav
Finished processing file:  Sound/fold1/157867-8-0-2.wav
Finished processing file:  Sound/fold1/151977-0-0-1.wav
Finished processing file:  Sound/fold1/119455-5-0-4.wav
Finished processing file:  Sound/fold1/78360-4

Finished processing file:  Sound/fold1/165067-2-0-112.wav
Finished processing file:  Sound/fold1/180937-7-3-1.wav
Finished processing file:  Sound/fold1/51022-3-7-0.wav
Finished processing file:  Sound/fold1/191431-9-0-73.wav
Finished processing file:  Sound/fold1/192124-2-0-11.wav
Finished processing file:  Sound/fold1/9031-3-1-0.wav
Finished processing file:  Sound/fold1/157867-8-0-5.wav
Finished processing file:  Sound/fold1/151977-0-0-6.wav
Finished processing file:  Sound/fold1/57553-3-1-0.wav
Finished processing file:  Sound/fold1/119455-5-0-3.wav
Finished processing file:  Sound/fold1/197318-6-6-0.wav
Finished processing file:  Sound/fold1/193698-2-0-114.wav
Finished processing file:  Sound/fold1/137156-9-0-28.wav
Finished processing file:  Sound/fold1/180937-7-1-3.wav
Finished processing file:  Sound/fold1/174294-6-1-0.wav
Finished processing file:  Sound/fold1/172314-9-0-40.wav
Finished processing file:  Sound/fold1/176787-5-0-22.wav
Finished processing file:  Sound/fold1/7776

Finished processing file:  Sound/fold1/21684-9-0-12.wav
Finished processing file:  Sound/fold1/114587-3-0-5.wav
Finished processing file:  Sound/fold1/180937-7-1-9.wav
Finished processing file:  Sound/fold1/193394-3-0-10.wav
Finished processing file:  Sound/fold1/159738-8-0-17.wav
Finished processing file:  Sound/fold1/78360-4-0-5.wav
Finished processing file:  Sound/fold1/176787-5-0-14.wav
Finished processing file:  Sound/fold1/177621-0-0-93.wav
Finished processing file:  Sound/fold1/176787-5-0-28.wav
Finished processing file:  Sound/fold1/196065-2-0-0.wav
Finished processing file:  Sound/fold1/124489-9-0-6.wav
Finished processing file:  Sound/fold1/193698-2-0-58.wav
Finished processing file:  Sound/fold1/57320-0-0-7.wav
Finished processing file:  Sound/fold1/193698-2-0-64.wav
Finished processing file:  Sound/fold1/99180-9-0-48.wav
Finished processing file:  Sound/fold1/7383-3-0-1.wav
Finished processing file:  Sound/fold1/176714-2-0-40.wav
Finished processing file:  Sound/fold1/14776

...

Finished processing file:  Sound/fold10/118278-4-0-6.wav
Finished processing file:  Sound/fold10/138017-9-1-10.wav
Finished processing file:  Sound/fold10/25037-6-0-0.wav
Finished processing file:  Sound/fold10/73524-0-0-99.wav
Finished processing file:  Sound/fold10/187863-4-0-0.wav
Finished processing file:  Sound/fold10/187110-2-0-10.wav
Finished processing file:  Sound/fold10/83502-0-0-9.wav
Finished processing file:  Sound/fold10/30344-3-0-4.wav
Finished processing file:  Sound/fold10/102857-5-0-30.wav
Finished processing file:  Sound/fold10/39847-5-0-4.wav
Finished processing file:  Sound/fold10/102857-5-0-4.wav
Finished processing file:  Sound/fold10/162134-7-13-1.wav
Finished processing file:  Sound/fold10/171478-9-0-58.wav
Finished processing file:  Sound/fold10/102857-5-0-24.wav
Finished processing file:  Sound/fold10/74364-8-1-22.wav
Finished processing file:  Sound/fold10/102857-5-0-18.wav
Finished processing file:  Sound/fold10/207213-2-0-52.wav
Finished processing file:  

Finished processing file:  Sound/fold10/138017-9-1-16.wav
Finished processing file:  Sound/fold10/129750-2-0-4.wav
Finished processing file:  Sound/fold10/118278-4-0-0.wav
Finished processing file:  Sound/fold10/162134-7-15-1.wav
Finished processing file:  Sound/fold10/199261-3-0-0.wav
Finished processing file:  Sound/fold10/74364-8-1-24.wav
Finished processing file:  Sound/fold10/66115-1-0-0.wav
Finished processing file:  Sound/fold10/73524-0-0-74.wav
Finished processing file:  Sound/fold10/102857-5-0-22.wav
Finished processing file:  Sound/fold10/74364-8-1-18.wav
Finished processing file:  Sound/fold10/39847-5-0-2.wav
Finished processing file:  Sound/fold10/102857-5-0-2.wav
Finished processing file:  Sound/fold10/119067-0-0-2.wav
Finished processing file:  Sound/fold10/164194-2-0-11.wav
Finished processing file:  Sound/fold10/73524-0-0-2.wav
Finished processing file:  Sound/fold10/15544-5-0-9.wav
Finished processing file:  Sound/fold10/162134-7-11-5.wav
Finished processing file:  Sou

Finished processing file:  Sound/fold10/15544-5-0-7.wav
Finished processing file:  Sound/fold10/99192-4-0-30.wav
Finished processing file:  Sound/fold10/99192-4-0-18.wav
Finished processing file:  Sound/fold10/100648-1-0-0.wav
Finished processing file:  Sound/fold10/27070-2-0-2.wav
Finished processing file:  Sound/fold10/26173-8-0-0.wav
Finished processing file:  Sound/fold10/74364-8-1-16.wav
Finished processing file:  Sound/fold10/167464-0-0-4.wav
Finished processing file:  Sound/fold10/73524-0-0-52.wav
Finished processing file:  Sound/fold10/208652-8-2-0.wav
Finished processing file:  Sound/fold10/146244-4-0-0.wav
Finished processing file:  Sound/fold10/102857-5-0-10.wav
Finished processing file:  Sound/fold10/117889-9-0-30.wav
Finished processing file:  Sound/fold10/187110-2-0-30.wav
Finished processing file:  Sound/fold10/83502-0-0-1.wav
Finished processing file:  Sound/fold10/202334-9-0-137.wav
Finished processing file:  Sound/fold10/115418-9-0-20.wav
Finished processing file:  So

Finished processing file:  Sound/fold10/187110-2-0-34.wav
Finished processing file:  Sound/fold10/102857-5-0-28.wav
Finished processing file:  Sound/fold10/167464-0-0-0.wav
Finished processing file:  Sound/fold10/74364-8-1-12.wav
Finished processing file:  Sound/fold10/102857-5-0-8.wav
Finished processing file:  Sound/fold10/39847-5-0-8.wav
Finished processing file:  Sound/fold10/102857-5-0-14.wav
Finished processing file:  Sound/fold10/15544-5-0-3.wav
Finished processing file:  Sound/fold10/155262-2-0-15.wav
Finished processing file:  Sound/fold10/164377-9-0-16.wav
Finished processing file:  Sound/fold10/99192-4-0-20.wav
Finished processing file:  Sound/fold10/181624-4-8-0.wav
Finished processing file:  Sound/fold10/81791-3-0-1.wav
Finished processing file:  Sound/fold10/167464-0-0-23.wav
Finished processing file:  Sound/fold10/73524-0-0-8.wav
Finished processing file:  Sound/fold10/167464-0-0-22.wav
Finished processing file:  Sound/fold10/27070-2-0-7.wav
Finished processing file:  So

Finished processing file:  Sound/fold10/162134-7-6-0.wav
Finished processing file:  Sound/fold10/99192-4-0-41.wav
Finished processing file:  Sound/fold10/118558-5-0-0.wav
Finished processing file:  Sound/fold10/14468-3-5-0.wav
Finished processing file:  Sound/fold10/84249-9-0-6.wav
Finished processing file:  Sound/fold10/178261-7-2-0.wav
Finished processing file:  Sound/fold10/165166-8-0-6.wav
Finished processing file:  Sound/fold10/178826-2-0-28.wav
Finished processing file:  Sound/fold10/83195-9-0-2.wav
Finished processing file:  Sound/fold10/178826-2-0-14.wav
Finished processing file:  Sound/fold10/118558-5-2-2.wav
Finished processing file:  Sound/fold10/73524-0-0-23.wav
Finished processing file:  Sound/fold10/93567-8-0-10.wav
Finished processing file:  Sound/fold10/88121-8-0-3.wav
Finished processing file:  Sound/fold10/188813-7-2-0.wav
Finished processing file:  Sound/fold10/180127-4-0-20.wav
Finished processing file:  Sound/fold10/81791-3-1-0.wav
Finished processing file:  Sound/